## How do small habitat patches support higher biodiversity for an equivalent habitat area?

Erin Ricaloglu, Jurek Kolasa

In [ ]:
#import packages
import numpy as np
import random
import math
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import seaborn as sns
import os
import glob
import statsmodels.api as sm
from scipy.stats import pearsonr, f_oneway

### Sorting through the CSV file. 

In [ ]:

# Define the directory where your CSV files are stored
directory_path = 'xxx'  

## NetLogo, has a code that will store a CSV file following each simulation run, with a designated location. This is outlined in the Code, within the application, and must be altered to each individual's desktop. 

# Load each CSV file into a DataFrame and append to the list
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):  # Only process CSV files
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path)
        df_list.append(df)

# Combine all DataFrames into one
df = pd.concat(df_list, ignore_index=True)

# Clean up column names by stripping extra spaces
df.columns = df.columns.str.strip()

# Ensure the 'Year' column is numeric
combined_data['Year'] = pd.to_numeric(combined_data['Year'], errors='coerce')
combined_data = combined_data.dropna(subset=['Year'])
combined_data['Year'] = combined_data['Year'].astype(int)

# Ensure 'Type A Count' and 'Type B Count' are numeric
combined_data['Type A Count'] = pd.to_numeric(combined_data['Type A Count'], errors='coerce').fillna(0)
combined_data['Type B Count'] = pd.to_numeric(combined_data['Type B Count'], errors='coerce').fillna(0)

# Sort data by Year and Patch for proper calculations
combined_data.sort_values(by=['Patch X', 'Patch Y', 'Year'], inplace=True)

# Calculate turnover for Type A (Specialists) and Type B (Generalists) per patch
combined_data['Type A Turnover'] = combined_data.groupby(['Patch X', 'Patch Y'])['Type A Count'].diff().abs().fillna(0)
combined_data['Type B Turnover'] = combined_data.groupby(['Patch X', 'Patch Y'])['Type B Count'].diff().abs().fillna(0)

# Group by Year to calculate the mean turnover per year for specialists and generalists
mean_turnover_per_year = combined_data.groupby('Year').agg({
    'Type A Turnover': 'mean',
    'Type B Turnover': 'mean'
})

# Function to convert string representation of a list to an actual list
def parse_list_from_string(string):
    try:
        # Remove brackets, quotes, and any extra spaces or characters
        cleaned_string = string.strip('[]').replace("'", "").replace("[", "").replace("]", "")
        # Split by comma and further clean each item
        items = cleaned_string.split(',')
        parsed_list = [item.strip() for item in items if item.strip()]
        return parsed_list
    except Exception as e:
        return []


## Modelling Species Turnover, for compositional change

In [ ]:
# Plot the mean species turnover over time for specialists and generalists
plt.figure(figsize=(10, 6))
plt.plot(mean_turnover_per_year.index, mean_turnover_per_year['Type A Turnover'], 
         marker='o', color='blue', linestyle='-', label='Mean Specialist Turnover')

plt.plot(mean_turnover_per_year.index, mean_turnover_per_year['Type B Turnover'], 
         marker='s', color='green', linestyle='-', label='Mean Generalist Turnover')

plt.xlabel('Year')
plt.ylabel('Mean Species Turnover')
plt.title('Mean Specialist vs. Generalist Turnover Over Time in Small Patches')
plt.legend()
plt.grid(visible=True, linestyle='--', alpha=0.5)
plt.show()

## Modelling Species Richness

In [ ]:
# Function to convert string representation of a list to an actual list
def parse_list_from_string(string):
    try:
        cleaned_string = string.strip('[]').replace("'", "").replace("[", "").replace("]", "")
        items = cleaned_string.split(',')
        parsed_list = [item.strip() for item in items if item.strip()]
        return parsed_list
    except Exception as e:
        return []

# Function to extract unique species from 'Current Species List'
def extract_species(species_list_str):
    species_list = parse_list_from_string(species_list_str)
    return set(species_list)  # Return as a set for uniqueness

# Apply the function to extract species per patch
df["Unique Species"] = df["Current Species List"].apply(extract_species)

# Separate species richness for specialists (Type A) and generalists (Type B)
df["Unique Specialist Species"] = df.apply(lambda row: set(row["Unique Species"]) if row["Type A Count"] > 0 else set(), axis=1)
df["Unique Generalist Species"] = df.apply(lambda row: set(row["Unique Species"]) if row["Type B Count"] > 0 else set(), axis=1)

# Group by 'Year' and calculate species richness (number of unique species per year)
specialist_richness_per_year = df.groupby("Year")["Unique Specialist Species"].apply(lambda species_sets: len(set().union(*species_sets)))
generalist_richness_per_year = df.groupby("Year")["Unique Generalist Species"].apply(lambda species_sets: len(set().union(*species_sets)))

# Plot specialist vs generalist species richness over time
plt.figure(figsize=(10, 6))
plt.plot(specialist_richness_per_year.index, specialist_richness_per_year.values, marker="o", color="blue", linestyle="-", label="Specialist Species Richness")
plt.plot(generalist_richness_per_year.index, generalist_richness_per_year.values, marker="s", color="green", linestyle="-", label="Generalist Species Richness")

plt.xlabel("Year")
plt.ylabel("Number of Unique Species (Richness)")
plt.title("Specialist vs Generalist Species Richness Over Time in Small Patches")
plt.legend()
plt.grid(visible=True, linestyle="--", alpha=0.5)
plt.show()

### This section will display numerical metrics, in regards to the loaded dataframe. Typically for an entire experiment and singular patch size. Ex. 30 runs and Small Patches. 

In [ ]:
# Load and Clean CSV Files
def load_and_clean_data(folder_path):
    df_list = []
    
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path, encoding="ISO-8859-1")
            df.columns = df.columns.str.strip()  # Clean column names
            
            # Convert list-like strings into actual lists
            if "Current Species List" in df.columns:
                df["Current Species List"] = df["Current Species List"].apply(parse_list_from_string)
            
            df_list.append(df)

    df_combined = pd.concat(df_list, ignore_index=True)
    
    # Print available columns for debugging
    print("\nAvailable columns in dataset:", df_combined.columns.tolist())

    return df_combined

# Convert String Representation of Lists
def parse_list_from_string(string):
    if isinstance(string, str):
        cleaned_string = string.strip('[]').replace("'", "").replace("[", "").replace("]", "")
        items = cleaned_string.split(',')
        return [item.strip() for item in items if item.strip()]
    return string

# Calculate Alpha Diversity (Species Richness per Patch)
def calculate_alpha_diversity(df):
    if "Current Species List" in df.columns:
        df["Species_Richness"] = df["Current Species List"].apply(len)
        return df.groupby(["Patch X", "Patch Y"])["Species_Richness"].mean()
    else:
        print("\nSkipping Alpha Diversity: 'Current Species List' column missing.")
        return None

# Calculate Gamma Diversity (Total Species Richness)
def calculate_gamma_diversity(df):
    if "Current Species List" in df.columns:
        all_species = set(species for sublist in df["Current Species List"] for species in sublist)
        return len(all_species)
    else:
        print("\nSkipping Gamma Diversity: 'Current Species List' column missing.")
        return None

# Calculate Turnover Percentage
def calculate_turnover_percentage(df):
    if "Turnover" in df.columns:
        df["Turnover_Percentage"] = (df["Turnover"] / df["Turnover"].sum()) * 100
    return df


# Perform ANOVA (Differences in Species Richness Across Patch Suitability)
def perform_anova(df):
    if "Patch Suitability" in df.columns and "Species_Richness" in df.columns:
        anova_result = f_oneway(
            df[df["Patch Suitability"] < df["Patch Suitability"].quantile(0.33)]["Species_Richness"],
            df[df["Patch Suitability"].between(df["Patch Suitability"].quantile(0.33), df["Patch Suitability"].quantile(0.66))]["Species_Richness"],
            df[df["Patch Suitability"] > df["Patch Suitability"].quantile(0.66)]["Species_Richness"]
        )
        print("\nANOVA Results for Species Richness by Patch Suitability:", anova_result)
        return anova_result
    else:
        print("\nSkipping ANOVA: Required columns missing.")
        return None


# Calculate Coefficient of Variation (CV)
def coefficient_of_variation(series):
    return (np.std(series) / np.mean(series)) * 100 if np.mean(series) != 0 else 0

def calculate_coefficient_of_variation(df):
    if "Species_Richness" in df.columns and "Turnover" in df.columns:
        cv_species_richness = coefficient_of_variation(df["Species_Richness"])
        cv_turnover = coefficient_of_variation(df["Turnover"])
        print("\nCoefficient of Variation (Species Richness):", cv_species_richness)
        print("Coefficient of Variation (Turnover):", cv_turnover)
        return cv_species_richness, cv_turnover
    else:
        print("\nSkipping CV Calculation: Required columns missing.")
        return None, None

# Main Execution Function
def main(folder_path):
    # Load and clean data
    df = load_and_clean_data(folder_path)

    # Compute Metrics
    alpha_diversity = calculate_alpha_diversity(df)
    gamma_diversity = calculate_gamma_diversity(df)
    df = calculate_turnover_percentage(df)

    # Perform Statistical Analyses
    anova_result = perform_anova(df)
    cv_species_richness, cv_turnover = calculate_coefficient_of_variation(df)

    # Print key metrics
    if alpha_diversity is not None:
        print("\nAlpha Diversity (Mean Species Richness per Patch):")
        print(alpha_diversity)
    if gamma_diversity is not None:
        print("\nGamma Diversity (Total Species Richness Across All Patches):", gamma_diversity)

# Run the script with folder path
folder_path = ""  # Replace with actual path
main(folder_path)

## Displaying the Bray-Curtis Dissimilarity

In [ ]:
sns.histplot(data=bray_curtis_df, x="Bray-Curtis Dissimilarity", binwidth=0.1)
plt.show()

## Turnover Stability Trends: Comparisons for Multiple Patch Sizes

### Define the two different directories (For ex. Small and Large Patch sizes, 30 runs)

In [ ]:
# Define directories
small_patch_dir = ''
large_patch_dir = ''

# Function to load and concatenate CSVs
def load_data(directory):
    df_list = []
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            file_path = os.path.join(directory, filename)
            df = pd.read_csv(file_path)
            df.columns = df.columns.str.strip()  # Ensure clean column names
            df_list.append(df)
    return pd.concat(df_list, ignore_index=True) if df_list else None

# Load small and large patch data
df_small = load_data(small_patch_dir)
df_large = load_data(large_patch_dir)

# Check if data loaded correctly
if df_small is None or df_large is None:
    print("Error loading data. Check directory paths.")

### Further CSV cleanup, to ensure files are read correctly. Further, year 1 is removed, as only years 2-5 are analyzed in the final results.

In [ ]:
# Remove leading and trailing spaces from column names
df_small.columns = df_small.columns.str.strip()
df_large.columns = df_large.columns.str.strip()

# Check again after cleaning
print("Cleaned Small Patch Dataset Columns:", df_small.columns)
print("Cleaned Large Patch Dataset Columns:", df_large.columns)

# Now you can select the columns safely
selected_columns = ['Turnover', 'Type A Count', 'Type B Count', 'Total Species']
df_small = df_small[selected_columns]
df_large = df_large[selected_columns]

# Strip spaces from column names
df_small.columns = df_small.columns.str.strip()
df_large.columns = df_large.columns.str.strip()

# Ensure 'Year' column is numeric
df_small["Year"] = df_small["Year"].astype(int)
df_large["Year"] = df_large["Year"].astype(int)

# Filter data: Keep only Years 2-5
df_small_filtered = df_small[df_small["Year"].between(2, 5)]
df_large_filtered = df_large[df_large["Year"].between(2, 5)]

# Group by Year to calculate mean turnover
small_turnover = df_small_filtered.groupby("Year")["Turnover"].mean()
large_turnover = df_large_filtered.groupby("Year")["Turnover"].mean()

# Create the plot
plt.figure(figsize=(10, 6))

# Line plots for turnover trends
sns.lineplot(x=small_turnover.index, y=small_turnover, label="Small Patches", color="blue", linewidth=2)
sns.lineplot(x=large_turnover.index, y=large_turnover, label="Large Patches", color="red", linewidth=2)

# Labels and formatting
plt.xlabel("Time (Years)", fontsize=14)
plt.ylabel("Mean Turnover", fontsize=14)
plt.title("Turnover Stability Trends Over Time (Years 2-5)", fontsize=16, fontweight="bold")
plt.legend(fontsize=12)
plt.grid(True, linestyle="--", alpha=0.6)
sns.set_style("whitegrid")
plt.rcParams["font.family"] = "DejaVu Serif"  # Elegant and readable

# Show plot
plt.show()

## Creating the Final Figures:

Figure 1: Scatter plots with polynomial regression lines depicting the relationship between total patch density (N) and species richness for specialists (red) and generalists (blue) in large (top row) and small (bottom row) patches. Top Left: In large patches, specialist richness follows a weak unimodal trend, peaking at intermediate densities before declining (R² = 0.41). Top Right: Generalist richness in large patches increases with total density, showing a strong positive correlation (R² = 0.70). Bottom Left: In small patches, specialist richness exhibits a clearer unimodal trend, increasing initially but declining at high densities, indicating possible competition effects (R² = 0.82). Bottom Right: Generalist richness in small patches increases sharply with total patch density, following an exponential-like trend (R² = 0.86). 

Figure 2: Created in Statistica. 


Figure 3: Temporal changes in generalist and specialist richness (black lines) and species turnover (red dashed lines) in large and small habitat patches from Years 2 to 5. Top Left: Generalist richness and Turnover in Large Patches. Top Right: Specialist richness and turnover in Large Patches.  Bottom Left: Generalist richness and turnover in Small Patches.  Bottom Right: Specialist richness with turnover in Small Patches.

## Figure 1

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Strip spaces from column names
df_small.columns = df_small.columns.str.strip()
df_large.columns = df_large.columns.str.strip()

# Convert necessary columns to numeric
numeric_cols = ["Type A Count", "Type B Count", "Patch Density"]

for col in numeric_cols:
    df_small[col] = pd.to_numeric(df_small[col], errors="coerce")
    df_large[col] = pd.to_numeric(df_large[col], errors="coerce")

# Filter data: Keep only Years 2-5
df_small_filtered = df_small[df_small["Year"].between(2, 5)]
df_large_filtered = df_large[df_large["Year"].between(2, 5)]

# Group by total patch density and compute mean richness
df_small_grouped = df_small_filtered.groupby("Patch Density")[["Type A Count", "Type B Count"]].mean().reset_index()
df_large_grouped = df_large_filtered.groupby("Patch Density")[["Type A Count", "Type B Count"]].mean().reset_index()

# Function to fit and plot polynomial regression with R² value in legend
def plot_polynomial_regression(ax, x_data, y_data, color, label):
    """Fits a quadratic regression model and plots with R² value in the legend."""
    x_data = x_data.values.reshape(-1, 1)
    y_data = y_data.values

    # Create polynomial features (quadratic)
    poly = PolynomialFeatures(degree=2)
    x_poly = poly.fit_transform(x_data)

    # Fit polynomial regression model
    model = LinearRegression().fit(x_poly, y_data)
    y_pred = model.predict(x_poly)
    
    # Calculate R² value
    r2 = r2_score(y_data, y_pred)
    
    # Sort x values for smooth plotting
    sorted_indices = np.argsort(x_data.flatten())
    x_sorted = x_data.flatten()[sorted_indices]
    y_pred_sorted = y_pred[sorted_indices]

    # Plot scatter and regression line
    ax.scatter(x_data, y_data, color=color, s=50, alpha=0.6, label=f"{label} Data")
    ax.plot(x_sorted, y_pred_sorted, linestyle="dashed", linewidth=2, color=color, 
            label=f"{label} Fit (R²={r2:.2f})")

    # Remove gridlines
    ax.grid(False)

    # Add legend to each subplot
    ax.legend(fontsize=10, loc="upper left")

    return r2

# Create figure with 2x2 subplots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

### **Large Patches: Specialist Richness vs. Total Density**
plot_polynomial_regression(axes[0, 0], df_large_grouped["Patch Density"], df_large_grouped["Type A Count"], "red", "Specialist")
axes[0, 0].set_title("Large Patches - Specialist Richness vs. Total Density", fontsize=14, fontweight="bold")
axes[0, 0].set_xlabel("Total Patch Density (N)", fontsize=12)
axes[0, 0].set_ylabel("Specialist Richness", fontsize=12)

### **Large Patches: Generalist Richness vs. Total Density**
plot_polynomial_regression(axes[0, 1], df_large_grouped["Patch Density"], df_large_grouped["Type B Count"], "blue", "Generalist")
axes[0, 1].set_title("Large Patches - Generalist Richness vs. Total Density", fontsize=14, fontweight="bold")
axes[0, 1].set_xlabel("Total Patch Density (N)", fontsize=12)
axes[0, 1].set_ylabel("Generalist Richness", fontsize=12)

### **Small Patches: Specialist Richness vs. Total Density**
plot_polynomial_regression(axes[1, 0], df_small_grouped["Patch Density"], df_small_grouped["Type A Count"], "red", "Specialist")
axes[1, 0].set_title("Small Patches - Specialist Richness vs. Total Density", fontsize=14, fontweight="bold")
axes[1, 0].set_xlabel("Total Patch Density (N)", fontsize=12)
axes[1, 0].set_ylabel("Specialist Richness", fontsize=12)

### **Small Patches: Generalist Richness vs. Total Density**
plot_polynomial_regression(axes[1, 1], df_small_grouped["Patch Density"], df_small_grouped["Type B Count"], "blue", "Generalist")
axes[1, 1].set_title("Small Patches - Generalist Richness vs. Total Density", fontsize=14, fontweight="bold")
axes[1, 1].set_xlabel("Total Patch Density (N)", fontsize=12)
axes[1, 1].set_ylabel("Generalist Richness", fontsize=12)

# Adjust layout for better spacing
plt.tight_layout()
plt.savefig("poly_regression.pdf", format="pdf")
plt.show()

## Figure 3

In [ ]:
# Set seaborn style and font preferences
sns.set_style("whitegrid")
plt.rcParams["font.family"] = "DejaVu Serif"  # Use a readable serif font

# Ensure numerical columns are properly formatted
numeric_cols = ["Turnover", "Type A Count", "Type B Count"]

# Convert columns to numeric
for col in numeric_cols:
    df_small[col] = pd.to_numeric(df_small[col], errors="coerce")
    df_large[col] = pd.to_numeric(df_large[col], errors="coerce")

# Filter Data for Years 2-5
df_small_filtered = df_small[(df_small["Year"] >= 2) & (df_small["Year"] <= 5)]
df_large_filtered = df_large[(df_large["Year"] >= 2) & (df_large["Year"] <= 5)]

# Group by year and compute mean richness & turnover
df_large_grouped = df_large_filtered.groupby("Year")[numeric_cols].mean()
df_small_grouped = df_small_filtered.groupby("Year")[numeric_cols].mean()

# Find y-axis limits for large patches
large_y_min, large_y_max = df_large_grouped.min().min(), df_large_grouped.max().max()

# Find y-axis limits for small patches (smaller range for clarity)
small_y_min, small_y_max = df_small_grouped.min().min(), df_small_grouped.max().max() * 1.2  

# Create the figure with 2x2 subplots
fig, axes = plt.subplots(2, 2, figsize=(14, 12), sharex=True)

# Define line properties
line_styles = {"Turnover": {"color": "red", "marker": "o", "linestyle": "--"},
               "Type A Count": {"color": "black", "marker": "s", "linestyle": "-"},
               "Type B Count": {"color": "black", "marker": "^", "linestyle": "-"}}

# Large Patches - Generalist Richness vs. Turnover
sns.lineplot(x=df_large_grouped.index, y=df_large_grouped["Turnover"], 
             **line_styles["Turnover"], label="Turnover", ax=axes[0, 0])
sns.lineplot(x=df_large_grouped.index, y=df_large_grouped["Type B Count"], 
             **line_styles["Type B Count"], label="Generalist Richness", ax=axes[0, 0])
axes[0, 0].set_title("Large Patches - Generalist Richness vs. Turnover", fontsize=15, fontweight="bold")
axes[0, 0].set_xlabel("Year", fontsize=13)
axes[0, 0].tick_params(axis="both", labelsize=12)
axes[0, 0].set_ylim(large_y_min, large_y_max)
axes[0, 0].legend(fontsize=12, frameon=True)

# Large Patches - Specialist Richness vs. Turnover
sns.lineplot(x=df_large_grouped.index, y=df_large_grouped["Turnover"], 
             **line_styles["Turnover"], label="Turnover", ax=axes[0, 1])
sns.lineplot(x=df_large_grouped.index, y=df_large_grouped["Type A Count"], 
             **line_styles["Type A Count"], label="Specialist Richness", ax=axes[0, 1])
axes[0, 1].set_title("Large Patches - Specialist Richness vs. Turnover", fontsize=15, fontweight="bold")
axes[0, 1].set_xlabel("Year", fontsize=13)
axes[0, 1].tick_params(axis="both", labelsize=12)
axes[0, 1].set_ylim(large_y_min, large_y_max)
axes[0, 1].legend(fontsize=12, frameon=True)

# Small Patches - Generalist Richness vs. Turnover (Zoomed Y-Axis)
sns.lineplot(x=df_small_grouped.index, y=df_small_grouped["Turnover"], 
             **line_styles["Turnover"], label="Turnover", ax=axes[1, 0])
sns.lineplot(x=df_small_grouped.index, y=df_small_grouped["Type B Count"], 
             **line_styles["Type B Count"], label="Generalist Richness", ax=axes[1, 0])
axes[1, 0].set_title("Small Patches - Generalist Richness vs. Turnover", fontsize=15, fontweight="bold")
axes[1, 0].set_xlabel("Year", fontsize=13)
axes[1, 0].tick_params(axis="both", labelsize=12)
axes[1, 0].set_ylim(small_y_min, small_y_max)  
axes[1, 0].legend(fontsize=12, frameon=True)

# Small Patches - Specialist Richness vs. Turnover (Zoomed Y-Axis)
sns.lineplot(x=df_small_grouped.index, y=df_small_grouped["Turnover"], 
             **line_styles["Turnover"], label="Turnover", ax=axes[1, 1])
sns.lineplot(x=df_small_grouped.index, y=df_small_grouped["Type A Count"], 
             **line_styles["Type A Count"], label="Specialist Richness", ax=axes[1, 1])
axes[1, 1].set_title("Small Patches - Specialist Richness vs. Turnover", fontsize=15, fontweight="bold")
axes[1, 1].set_xlabel("Year", fontsize=13)
axes[1, 1].tick_params(axis="both", labelsize=12)
axes[1, 1].set_ylim(small_y_min, small_y_max)  
axes[1, 1].legend(fontsize=12, frameon=True)

# Apply the same y-axis label for all plots
for ax in axes.flat:
    ax.set_ylabel("Turnover and Species Count", fontsize=13)

# Adjust layout for better spacing
plt.tight_layout()
plt.show()